**Κώδικας visualization του Banners Tag Localization**

Με τον παρακάτω κώδικα θα κάνουμε visualize τα δεδομένα. Συγκεκριμένα, έχουμε visualization της φάσης σε συνάρτηση με την απόσταση κίνησης r, καθώς και ιστόγραμμα από τη χρονική στιγμή δειγματοληψίας για κάθε tag. Για το ιστόγραμμα πρώτα αφαιρούμε τις ακραίες τιμές. Παρατηρούμε ότι ακολουθεί κανονική κατανομή. Επίσης αποθηκεύουμε τα δεδομένα σε μορφή xl για να είναι πιο εύκολο να τα δει κανείς στο μέλλον. Τέλος, έχουμε γράψει κώδικα για visualization σε 3D και ως προς x και y, αλλά στην πραγματικότητα αυτό δεν μας απασχολεί και δεν δίνει κάποια πρακτική βοήθεια, οπότε το αφήσαμε εκτός.

Προσοχή! Ο κώδικας αποθηκεύει στον υπολογιστή τα αρχεία σε καινούριους φακέλους. Αν οι φάκελοι αυτοί υπάρχουν ήδη, καλό είναι να μην τον ξανατρέξετε και απλά να τους ανοίξετε και να δείτε τα αποτελέσματα επιτόπου. Το σκριπτάκι δίνεται μόνο για λόγους πληρότητας. Δεν χρείαζεται να ξανατρέξει.

Σε περίπτωση που πρέπει να τρέξει, το version της Python είναι 3.11.9 και οι βιβλιοθήκες matplotlib,pandas,numpy πρέπει να έχουν εγκατασταθεί στο σύστημα. Το script πρέπει να τρέξει σε directory που να περιέχει τον φάκελο Experiments.

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import os, shutil
import numpy as np


  

def format_files_type_1(files,path_of_txt,output_path_img,output_path_xl):
    for file in files:
        data = pd.read_csv(f"{path_of_txt}/{file}")
        #χρονική στιγμή που πάρθηκε η μετρηση (seconds), νούμερο κεραίας, φάση του σήματος, ισχύς του σήματος, 
        #x' συν/νη της κεραίας εκείνη τη χρονική στιγμή (cm), y' συν/νη της κεραίας (cm), z' συν/νη της κεραίας (cm), προσανατολισμός της κεραίας
        data.columns = ["timestamp (sec)","Antenna","Phase","Power","x(cm)","y(cm)","z(cm)","Antenna orientation"]
        filename = file[1:-4]
        
        Delta_x = data.values[1:,4]-data.values[0:-1,4]
        Delta_y = data.values[1:,5]-data.values[0:-1,5]
        Delta_r = np.sqrt(np.pow(Delta_x,2)+np.pow(Delta_y,2))
        if(len(Delta_r) > 1):
            Q1 = np.percentile(Delta_r, 25)
            Q3 = np.percentile(Delta_r, 75)
            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            Delta_r_filtered = [x for x in Delta_r if lower_bound <= x <= upper_bound]

            
            fig5 = plt.figure()
            plt.hist(Delta_r_filtered,bins=6)
            plt.title(f"{filename} Delta r hist")
            plt.savefig(f"{output_path_img}/{filename}_Delay_hist.png")
            plt.close()

        r = np.cumsum(Delta_r)
        data["r(cm)"] = np.hstack((r,None))

        #save data to xl
        data.to_excel(f"{output_path_xl}/{filename}.xlsx")

        #create and save the figures
        # fig = plt.figure(figsize=(12, 12))
        # ax = fig.add_subplot(projection='3d')

        # ax.scatter(data.values[:,4],data.values[:,5],-data.values[:,2],s=0.5)
        # ax.set_title(f"{filename} 3D View")
        # ax.set_xlabel("X (cm)")
        # ax.set_ylabel("Y (cm)")
        # ax.set_zlabel("phase (rad)")
        # plt.savefig(f"{output_path_img}/{filename}_3D_view.png")

        # fig2 = plt.figure()
        # plt.scatter(data.values[:,4],-data.values[:,2],marker='o',s=0.5)
        # plt.title(f"{filename} X View")
        # plt.xlabel("X (cm)")
        # plt.ylabel("Phase (rad)")
        # plt.savefig(f"{output_path_img}/{filename}_X_view.png")


        # fig3 = plt.figure()
        # plt.scatter(data.values[:,5],-data.values[:,2],marker='o',s=0.5)
        # plt.title(f"{filename} Y View")
        # plt.xlabel("Y (cm)")
        # plt.ylabel("Phase (rad)")
        # plt.savefig(f"{output_path_img}/{filename}_Y_view.png")

        fig4 = plt.figure()
        plt.scatter(r,-data.values[:-1,2],marker='o',s=0.5)
        plt.title(f"{filename} R View")
        plt.xlabel("R (cm)")
        plt.ylabel("Phase (rad)")
        plt.savefig(f"{output_path_img}/{filename}_R_view.png")
        plt.close()

def format_files_type_2(files,path_of_txt_unwrapped,output_path_img_unwrapped,output_path_xl_unwrapped):
    for file in files:
        data = pd.read_csv(f"{path_of_txt_unwrapped}/{file}")
        #χρονική στιγμή που πάρθηκε η μετρηση (second), x' συν/νη της κεραίας εκείνη τη χρονική στιγμή (cm) στο χάρτη του ρομπότ,
        #y' συν/νη της κεραίας (cm), z' συν/νη της κεραίας (cm), ισχύς του σήματος, φάση του σήματος, unwrapped φάση
        data.columns = ["timestamp (sec)","x(cm)","y(cm)","z(cm)","P","phase(rad)","unwrapped phase(rad)"]
        filename = file[1:-4]
        
        Delta_x = data.values[1:,1]-data.values[0:-1,1]
        Delta_y = data.values[1:,2]-data.values[0:-1,2]
        Delta_r = np.sqrt(np.pow(Delta_x,2)+np.pow(Delta_y,2))
        if(len(Delta_r) > 1):
            Q1 = np.percentile(Delta_r, 25)
            Q3 = np.percentile(Delta_r, 75)
            IQR = Q3 - Q1

            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            Delta_r_filtered = [x for x in Delta_r if lower_bound <= x <= upper_bound]

            fig5 = plt.figure()
            plt.hist(Delta_r_filtered)
            plt.title(f"{filename} Delta r hist")
            plt.savefig(f"{output_path_img_unwrapped}/{filename}_Delay_hist.png")
            plt.close()

        r = np.cumsum(Delta_r)
        data["r(cm)"] = np.hstack((r,None))
        #print data to excel
        data.to_excel(f"{output_path_xl_unwrapped}/{filename}.xlsx")

        #make the figures and save them
        # fig = plt.figure(figsize=(12, 12))
        # ax = fig.add_subplot(projection='3d')
        # ax.scatter(data.values[:,1],data.values[:,2],data.values[:,6],s=0.5)
        # ax.set_title(f"{filename} 3D View")
        # ax.set_xlabel("X (cm)")
        # ax.set_ylabel("Y (cm)")
        # ax.set_zlabel("phase (rad)")
        # plt.savefig(f"{output_path_img_unwrapped}/{filename}_3D_view.png")

        # fig2 = plt.figure()
        # plt.scatter(data.values[:,1],data.values[:,6],marker='o',s=0.5)
        # plt.title(f"{filename} X View")
        # plt.xlabel("X (cm)")
        # plt.ylabel("Phase (rad)")
        # plt.savefig(f"{output_path_img_unwrapped}/{filename}_X_view.png")


        # fig3 = plt.figure()
        # plt.scatter(data.values[:,2],data.values[:,6],marker='o',s=0.5)
        # plt.title(f"{filename} Y View")
        # plt.xlabel("Y (cm)")
        # plt.ylabel("Phase (rad)")
        # plt.savefig(f"{output_path_img_unwrapped}/{filename}_Y_view.png")

        fig4 = plt.figure()
        plt.scatter(r,data.values[:-1,6],marker='o',s=0.5)
        plt.title(f"{filename} R View")
        plt.xlabel("R (cm)")
        plt.ylabel("Phase (rad)")
        plt.savefig(f"{output_path_img_unwrapped}/{filename}_R_view.png")
        plt.close()


def main():
    path_type = ["S_path" "Strainght_path" "V_path"]
    main_files = ["Single Banner Horizontal","Single Banner Vertical","Two Banners Back 2 Back","Two Banners Side by Side"]
    for main_file in main_files:
        path_types = os.listdir(f"Experiments/Banners Tag Localization/{main_file}")
        for path_type in path_types:
            antennas = os.listdir(f"Experiments/Banners Tag Localization/{main_file}/{path_type}")
            for antenna in antennas:
                if(antenna[0:7] == "Antenna"):    
                    path_of_txt = f"Experiments/Banners Tag Localization/{main_file}/{path_type}/{antenna}/measurements"
                    path_of_txt_unwrapped = f"Experiments/Banners Tag Localization/{main_file}/{path_type}/{antenna}/unwrapped_measurements"
                    output_path_xl = f"Experiments/Banners Tag Localization/{main_file}/{path_type}/{antenna}/measurements_xl"
                    output_path_xl_unwrapped = f"Experiments/Banners Tag Localization/{main_file}/{path_type}/{antenna}/measurements_xl_unwrapped"
                    output_path_img_unwrapped = f"Experiments/Banners Tag Localization/{main_file}/{path_type}/{antenna}/measurements_img_unwrapped"
                    output_path_img = f"Experiments/Banners Tag Localization/{main_file}/{path_type}/{antenna}/measurements_img"

                    #This code removes all the directories created and is meant to clean up the entire process
                    
                    # if os.path.isdir(output_path_img):
                    #     shutil.rmtree(output_path_img)
                    # if os.path.isdir(output_path_img_unwrapped):
                    #     shutil.rmtree(output_path_img_unwrapped)
                    # if os.path.isdir(output_path_xl):
                    #     shutil.rmtree(output_path_xl)
                    # if os.path.isdir(output_path_xl_unwrapped):
                    #     shutil.rmtree(output_path_xl_unwrapped)

                    #we create the directories that store the xl files and the images and render and save them inside.
                    os.makedirs(output_path_img, exist_ok=True)
                    os.makedirs(output_path_img_unwrapped,exist_ok=True)
                    os.makedirs(output_path_xl,exist_ok=True)
                    os.makedirs(output_path_xl_unwrapped,exist_ok=True)
                    if(os.path.exists(path_of_txt)):
                        files = os.listdir(path_of_txt)
                        format_files_type_1(files,path_of_txt,output_path_img,output_path_xl)
                    if(os.path.exists(path_of_txt_unwrapped)):
                        files = os.listdir(path_of_txt_unwrapped)  
                        format_files_type_2(files,path_of_txt_unwrapped,output_path_img_unwrapped,output_path_xl_unwrapped)

if __name__  == "__main__":  
    main()